In [7]:
import nest_asyncio
from dbgpt.agent import (
    AgentContext,
    GptsMemory,
    LLMConfig,
    ResourceLoader,
    UserProxyAgent,
)
from dbgpt.agent.expand.code_assistant_agent import CodeAssistantAgent
from dbgpt.agent.plan import AutoPlanChatManager
from dbgpt.model.proxy import OpenAILLMClient

nest_asyncio.apply()

In [ ]:
# Set your api key and api base url
# os.environ["OPENAI_API_KEY"] = "Your API"
# os.environ["OPENAI_API_BASE"] = "https://api.openai.com/v1"

In [4]:
llm_client = OpenAILLMClient(model_alias="gpt-4")
context: AgentContext = AgentContext(conv_id="test456", gpts_app_name="代码分析助手")

default_memory = GptsMemory()

resource_loader = ResourceLoader()

coder = (
    await CodeAssistantAgent()
    .bind(context)
    .bind(LLMConfig(llm_client=llm_client))
    .bind(default_memory)
    .bind(resource_loader)
    .build()
)

manager = (
    await AutoPlanChatManager()
    .bind(context)
    .bind(default_memory)
    .bind(LLMConfig(llm_client=llm_client))
    .build()
)
manager.hire([coder])

user_proxy = await UserProxyAgent().bind(context).bind(default_memory).build()


await user_proxy.initiate_chat(
    recipient=manager,
    reviewer=user_proxy,
    message="Obtain simple information about issues in the repository 'eosphoros-ai/DB-GPT' in the past three days and analyze the data. Create a Markdown table grouped by day and status.",
    # message="Find papers on gpt-4 in the past three weeks on arxiv, and organize their titles, authors, and links into a markdown table",
    # message="find papers on LLM applications from arxiv in the last month, create a markdown table of different domains.",
)


--------------------------------------------------------------------------------
User (to PlanManager)-[]:

"Obtain simple information about issues in the repository 'eosphoros-ai/DB-GPT' in the past three days and analyze the data. Create a Markdown table grouped by day and status."

--------------------------------------------------------------------------------
un_stream ai response: [
  {
    "serial_number": "1",
    "agent": "CodeEngineer",
    "content": "Use Github API to retrieve issues from the repository 'eosphoros-ai/DB-GPT' in the past three days.",
    "rely": ""
  },
  {
    "serial_number": "2",
    "agent": "CodeEngineer",
    "content": "Analyze the retrieved issues data, extract the necessary information such as issue title, status, and created date.",
    "rely": "1"
  },
  {
    "serial_number": "3",
    "agent": "CodeEngineer",
    "content": "Group the extracted data by day and status.",
    "rely": "2"
  },
  {
    "serial_number": "4",
    "agent": "CodeEngine

GroupChat select_speaker failed to resolve the next speaker's name. This is because the speaker selection OAI call returned:
CodeEngineer



--------------------------------------------------------------------------------
PlanManager (to Turing)-[]:

"Use Github API to retrieve issues from the repository 'eosphoros-ai/DB-GPT' in the past three days."

--------------------------------------------------------------------------------
un_stream ai response: To retrieve issues from the GitHub repository 'eosphoros-ai/DB-GPT' in the past three days, we can use the GitHub REST API. Here is a Python script that accomplishes this. This script uses the 'requests' library to send a GET request to the GitHub API. The script then parses the JSON response to filter out issues created in the last three days.

```python
# python
import requests
from datetime import datetime, timedelta
import pytz

# Get the current date and time
now = datetime.now()

# Calculate the date 3 days ago
three_days_ago = now - timedelta(days=3)

# Convert the date to ISO 8601 format
three_days_ago_iso = three_days_ago.isoformat()

# Specify the URL of the GitHu

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


un_stream ai response: True

--------------------------------------------------------------------------------
Turing (to PlanManager)-[gpt-4]:

"To retrieve issues from the GitHub repository 'eosphoros-ai/DB-GPT' in the past three days, we can use the GitHub REST API. Here is a Python script that accomplishes this. This script uses the 'requests' library to send a GET request to the GitHub API. The script then parses the JSON response to filter out issues created in the last three days.\n\n```python\n# python\nimport requests\nfrom datetime import datetime, timedelta\nimport pytz\n\n# Get the current date and time\nnow = datetime.now()\n\n# Calculate the date 3 days ago\nthree_days_ago = now - timedelta(days=3)\n\n# Convert the date to ISO 8601 format\nthree_days_ago_iso = three_days_ago.isoformat()\n\n# Specify the URL of the GitHub API\nurl = \"https://api.github.com/repos/eosphoros-ai/DB-GPT/issues\"\n\n# Send a GET request to the GitHub API\nresponse = requests.get(url)\n\n# Check 

...


--------------------------------------------------------------------------------
PlanManager (to Turing)-[]:

"Read the result data of the dependent steps in the above historical message to complete the current goal:Create a Markdown table with the grouped data."

--------------------------------------------------------------------------------
un_stream ai response: To create a Markdown table with the grouped data, we can write a Python function that converts the data into a Markdown table format.

```python
# python code
def create_markdown_table(data):
    header = "| " + " | ".join(data.columns) + " |"
    separator = "| " + " | ".join("---" for _ in data.columns) + " |"
    rows = ["| " + " | ".join(str(x) for x in row) + " |" for row in data.values]
    return "\n".join([header, separator] + rows)

# Assuming the data is stored in a pandas DataFrame named 'df'
print(create_markdown_table(df))
```

Please replace the `df` with the actual DataFrame that contains your data. This fun

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change



--------------------------------------------------------------------------------
Turing (to PlanManager)-[gpt-4]:

"To create a Markdown table with the grouped data, we can write a Python function that converts the data into a Markdown table format.\n\n```python\n# python code\ndef create_markdown_table(data):\n    header = \"| \" + \" | \".join(data.columns) + \" |\"\n    separator = \"| \" + \" | \".join(\"---\" for _ in data.columns) + \" |\"\n    rows = [\"| \" + \" | \".join(str(x) for x in row) + \" |\" for row in data.values]\n    return \"\\n\".join([header, separator] + rows)\n\n# Assuming the data is stored in a pandas DataFrame named 'df'\nprint(create_markdown_table(df))\n```\n\nPlease replace the `df` with the actual DataFrame that contains your data. This function will output a Markdown table as a string. The table can be copied and pasted into any Markdown file or Markdown-supported text field."
>>>>>>>>Turing Review info: 
Pass(None)
>>>>>>>>Turing Action report: 
exec

execute_code was called without specifying a value for use_docker. Since the python docker package is not available, code will be run natively. Note: this fallback behavior is subject to change


un_stream ai response: True

--------------------------------------------------------------------------------
Turing (to PlanManager)-[gpt-4]:

"Apologies for the confusion. It seems there was a misunderstanding. The 'df' was a placeholder for your actual data. However, since there's no data provided in the context, I'll assume the data based on the previous responses and create a markdown table. \n\n```python\n# python code\nimport pandas as pd\n\n# Assuming the data is like this:\ndata = {\n    'created_date': ['2024-04-07', '2024-04-08', '2024-04-09', '2024-04-10'],\n    'status': ['Not specified', 'Not specified', 'Not specified', 'Not specified'],\n    'count': [1, 1, 3, 2]\n}\n\ndf = pd.DataFrame(data)\n\ndef create_markdown_table(data):\n    header = \"| \" + \" | \".join(data.columns) + \" |\"\n    separator = \"| \" + \" | \".join(\"---\" for _ in data.columns) + \" |\"\n    rows = [\"| \" + \" | \".join(str(x) for x in row) + \" |\" for row in data.values]\n    return \"\\n\"